# CS524:  Introduction to Optimization Lecture 28
## Michael C. Ferris <br> Computer Sciences Department<br>University of Wisconsin-Madison
## November 8, 2023

## Vector norms
We want to solve Ax=b, but there is no solution.  Define the residual to be the quantity $r:=b−Ax$.  We can’t make it zero, so instead we try to make it small.  Many options!
- minimize the largest component (a.k.a.  the ∞-norm)<br>
\begin{eqnarray*}
‖r‖_∞= \underset{i}{\text{max}}|r_i|
\end{eqnarray*}
- minimize the sum of absolute values (a.k.a. the 1-norm)<br>
\begin{eqnarray*}
‖r‖_1=|r_1|+|r_2|+···+|r_m|
\end{eqnarray*}
- minimize the Euclidean norm (a.k.a. the 2-norm)<br>
\begin{eqnarray*}
‖r‖_2=‖r‖=\sqrt{r_1^2+r_2^2+\dots+r_m^2}
\end{eqnarray*}
Equivalently, we minimize the square of this, a nice convex quadratic

## Vector norms
- minimizing the largest component is an LP:<br>
\begin{eqnarray*}
 \min_x \max_i | a_i^Tx−b_i | & \iff &
 \begin{aligned}
 & \min_{x,t} & & t \\
 & \text{s.t.} 
 & & −t \leq a_i^Tx−b_i \leq t
 \end{aligned}
\end{eqnarray*}
<br>
- minimizing the sum of absolute values is an LP:<br>
\begin{eqnarray*}
 \min_x
 \displaystyle\sum_{i=1}^{m} | a_i^Tx−b_i | & \iff & 
 \begin{aligned}
 & \min_{x,d_i} & & d_1+···+d_m \\
 & \text{s.t.} & & −d_i \leq a_i^Tx−b_i \leq d_i
 \end{aligned}
\end{eqnarray*}
<br>
- minimizing the 2-norm squared is not an LP!<br>
\begin{eqnarray*}
 \min_{x} 
 \displaystyle\sum_{i=1}^{m}(a_i^Tx−b_i)^2
\end{eqnarray*}

Can also formulate using difference of positive variables $a_i^Tx - b_i = p_i - n_i$, $p_i, n_i \geq 0$ but this is typically slower for the barrier method to solve (for large problems see leastsquares.gms model)

## Examples from least squares lecture slides

$L_{\infty}$:

In [1]:
%load_ext gams.magic
m = gams.exchange_container

In [2]:
%%gams
variables x, t;
equations m1, m2, m3, m4;

m1.. t =g= x - 1;
m2.. t =g= -(x-1);
m3.. t =g= x - 2;
m4.. t =g= -(x-2);

model linf /m1,m2,m3,m4/;
solve linf using lp min t;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),0.5,4,2,LP,CPLEX,0.016


In [3]:
xval = m['x'].toValue()
print(f'Optimal x ={xval:f}')

Optimal x =1.500000


$L_1$:

In [4]:
%%gams
variables d1, d2, d3, d4, obj;
equations n1, n2, n3, n4, o1;

n1.. d1 =g= x - 1;
n2.. d1 =g= -(x-1);
n3.. d2 =g= x - 2;
n4.. d2 =g= -(x-2);
o1.. obj =e= d1 + d2;

model lone /n1,n2,n3,n4,o1/;
solve lone using lp min obj;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1.0,5,4,LP,CPLEX,0.003


In [5]:
xval = m['x'].toValue()
print(f'Optimal x ={xval:f}')

Optimal x =2.000000


$L_2$:

In [6]:
%%gams
equations o2;

o2.. obj =e= sqr(x - 1) + sqr(x-2);

model l2 /o2/;
solve l2 using qcp min obj;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),0.5,1,2,QCP,CONOPT,0.023


In [7]:
xval = m['x'].toValue()
print(f'Optimal x ={xval:f}')

Optimal x =1.500000


In [8]:
gams.reset()
m = gams.exchange_container

## Another larger example  

First load in new data then do $L_\infty$, $L_1$, least squares models

In [9]:
%%gams
$title  Simple Linear Regression Model with different loss functions

set     i       Observations /i1*i100/,
        k       Parameters /k1*k2/;

parameter       b(i)    RHS values
                A(i,k)  Data points;

*  Generate a random dataset.
A(i,k) = uniform(0,2);

*  Randomly generate x0:
parameter  x0(k)   True value of X;
x0(k)   = uniform(0,2);

b(i) = sum(k,A(i,k)*x0(k)) + 0.1*normal(0,1);

parameter compare Comparison of estimates;
compare(k,"x0") = x0(k); 

In [10]:
%%gams
variables       x(k)    Unknown parameter estimates
                r(i)    Residual in equation i
                obj     Objective function;

equations boxnormUP, boxnormLO, rdef;

rdef(i)..       r(i) =e= sum(k, a(i,k)*x(k)) - b(i);

boxnormUP(i)..  OBJ =G= r(i);

boxnormLO(i)..  OBJ =G= -r(i);

model boxnorm/rdef,boxnormLO,boxnormUP/;
solve boxnorm using lp minimizing OBJ;
compare(k,"Linf") = x.l(k)

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),0.2376,300,103,LP,CPLEX,0.002


In [11]:
%%gams
equations absnormUP, absnormLO, objabs;

nonnegative variable d(i)  Deviation in the ith norm;

objabs..        OBJ =e= sum(i, d(i));

absnormUP(i)..  d(i) =G= r(i);

absnormLO(i)..  d(i) =G= -r(i);

model absnorm/rdef,absnormLO,absnormUP, objabs/;
solve absnorm using lp minimizing OBJ;
compare(k,"L1") = x.l(k)

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),8.1217,301,203,LP,CPLEX,0.005


In [12]:
%%gams

equations       objdef;

objdef..        obj =e= sum(i, sqr(r(i)));

model lsqr /objdef, rdef/;
solve lsqr using qcp minimizing OBJ;
compare(k,"L2") = x.l(k);

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),1.0007,101,103,QCP,CONOPT,0.007


In [13]:
display(m['compare'].pivot())

,x0,Linf,L1,L2
k1,1.865521,1.871575,1.864296,1.861592
k2,0.697531,0.716341,0.706407,0.705419


What about the k-norm?  (k largest absolute values of r)
See lecture notes and Gaudiso/Hiriart-Urruty paper
k = 1 gives inf norm
k = n gives L1 norm

In [14]:
%%gams
$set k 3

positive variables p(i), q(i), u(i), v(i), w;
equations defr(i), defn(i), defobj;

defobj..
  obj =e= sum(i, p(i) + q(i)) + %k%*w;

defr(i)..
  p(i) - q(i) + u(i) - v(i) =e= r(i);

defn(i)..
   -u(i) - v(i) + w =e= 0;

model knorm /rdef,defr,defn,defobj/;
solve knorm using lp min obj;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),0.6905,301,504,LP,CPLEX,0.003


In [15]:
# sum of 3 largest residuals
import numpy as np
display(np.sum(np.sort(np.abs(m['r'].records.level))[-3:]))

0.6904787865111004

In [16]:
display(m['x'].records[['k','level']])

,k,level,marginal,lower,upper,scale
0,k1,1.852687,0.0,-inf,inf,1.0
1,k2,0.705917,0.0,-inf,inf,1.0


## Parametric regression

We are given noisy data points $(z_i, y_i)$ (the training set).
- We suspect they are related by 
$$y\approx pz^2+qz+r =: \phi(z; x)$$ 
- Find $x = (p, q, r)$ so $\phi(z; x)$ best agrees with the data $y$.

Writing all the equations:
$$
\begin{array}{c}
y_1 \approx pz_1^2+qz_1+r \\
y_2 \approx pz_2^2+qz_2+r \\
\vdots \\
y_m \approx pz_m^2+qz_m+r 
\end{array}
\Rightarrow
      \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_m
      \end{bmatrix}
      \approx
      \begin{bmatrix} z_1^2 & z_1 & 1 \\ z_2^2 & z_2 & 1 \\ \vdots & \vdots & \vdots \\ 
        z_m^2 & z_m & 1 
      \end{bmatrix}
      \begin{bmatrix} p \\ q \\ r
      \end{bmatrix}
      =: A x
$$
- $a_i^T = (z_i^2, z_i, 1)$ for $i=1,\ldots,m$
- Curve fitting problem is also called parametric regression

## More complicated:

-  $y \approx p e^z + q \cos(z) - r \sqrt{z} + s z^3$
- Find $x = (p, q, r, s)$ that best agrees with the data
-  Writing all the equations:
$$
      \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_m
      \end{bmatrix}
      \approx
      \begin{bmatrix} e^{z_1} & \cos(z_1) & -\sqrt{z_1} & z_1^3 \\ e^{z_2} & \cos(z_2) & -\sqrt{z_2} & z_2^3 \\ \vdots & \vdots & \vdots & \vdots \\ 
        e^{z_m} & \cos(z_m) & -\sqrt{z_m} & z_m^3  
      \end{bmatrix}
      \begin{bmatrix} p \\ q \\ r \\ s 
      \end{bmatrix}
      =: A x
$$
- $a_i^T = (e^z_i, \cos(z_i), -\sqrt{z_i},  z_i^3)$ for $i=1,\ldots,m$; $\phi(z; x) = p e^z + q \cos(z) - r \sqrt{z} + s z^3$
- Still a linear least squares problem (data is nonlinear)

In [17]:
%gams_cleanup --closedown